# SVM localization for 5G NSA

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/LTEM_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAXMM1,BMAXMM2,BMAXMM3,BMBACM1,BMBACM2,BMBAKN1,BMBAKN2,...,BMZVON2,BMZVON3,TBTS1M1,TBTS3M1,TBTS3M2,TBTS5N1,TBTS5N2,TBTS5N3,TBTS6M1,TBTS6M2
0,1701853011,49.227137,16.575807,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1701853012,49.227134,16.575806,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1701853013,49.227133,16.575806,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1701853014,49.227133,16.575805,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1701853015,49.227132,16.575804,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28685,1701951192,49.230384,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28686,1701951194,49.230385,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28687,1701951196,49.230386,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28688,1701951198,49.230387,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [4]:
# Scaling X and y
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, shuffle=True, test_size=0.2)

## SVM

In [7]:
from sklearn.svm import SVR

In [8]:
regressor_lat = SVR(kernel='rbf')
regressor_lat.fit(X_train, y_train[:,0])

SVR()

In [9]:
regressor_lon = SVR(kernel='rbf')
regressor_lon.fit(X_train, y_train[:,1])

SVR()

In [10]:
y_pred_lat = regressor_lat.predict(X_test)
y_pred_lon = regressor_lon.predict(X_test)

In [11]:
y_pred_lat

array([ 0.90867493,  1.93325167, -0.08993415, ..., -0.56666518,
        0.02187129, -1.25670167])

In [12]:
y_pred_lon

array([ 0.44856818,  0.60567293, -0.42950387, ..., -0.99827974,
       -0.39308179,  0.27385857])

In [13]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [14]:
# R^2 score
score_lat = r2_score(y_test[:,0], y_pred_lat)

print("R^2 lat - {}%".format(round(score_lat, 5) *100))

R^2 lat - 98.153%


In [15]:
score_lon = r2_score(y_test[:,1], y_pred_lon)

print("R^2 lon - {}%".format(round(score_lon, 5) *100))

R^2 lon - 97.67%


In [16]:
score_combine = (score_lat + score_lon)/2

print("R^2 combine - {}%".format(round(score_combine, 5) *100))

R^2 combine - 97.912%


## MSE and RMSE

In [17]:
y_test_fin = sc_y.inverse_transform(y_test)
y_test_fin

array([[49.211255, 16.627573],
       [49.228392, 16.631154],
       [49.202274, 16.578587],
       ...,
       [49.186747, 16.572985],
       [49.196729, 16.591564],
       [49.176022, 16.621185]])

In [18]:
y_pred_join = np.stack((y_pred_lat, y_pred_lon), axis=1)
y_pred_join

array([[ 0.90867493,  0.44856818],
       [ 1.93325167,  0.60567293],
       [-0.08993415, -0.42950387],
       ...,
       [-0.56666518, -0.99827974],
       [ 0.02187129, -0.39308179],
       [-1.25670167,  0.27385857]])

In [19]:
y_pred_fin = sc_y.inverse_transform(y_pred_join)
y_pred_fin

array([[49.21224795, 16.62540654],
       [49.22911415, 16.63104117],
       [49.19580922, 16.59391407],
       ...,
       [49.18796145, 16.57351466],
       [49.19764972, 16.59522037],
       [49.17660233, 16.61914049]])

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mse_svm = mean_squared_error(y_test_fin, y_pred_fin)
rmse_svm = mean_squared_error(y_test_fin, y_pred_fin, squared = False)

print(mse_svm)
print(rmse_svm)

1.7638507254908565e-05
0.0038752063551167663


## Distance error

In [22]:
from math import acos, sin, cos, radians

# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist


In [23]:
geo_dist_svm = geo_distance(y_test_fin, y_pred_fin)
geo_dist_svm

array([0.19224044, 0.08071633, 1.32549765, ..., 0.14041993, 0.28471963,
       0.1620236 ])

In [24]:
geo_dist_svm_m = [i * 1000 for i in geo_dist_svm]

In [25]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_svm = Average(geo_dist_svm_m)

# Printing average of the list
print("Average SVM error [m] =", round(average_error_svm, 5))
print("Max SVM error [m] = ", np.max(geo_dist_svm_m))
print("Min SVM error [m] = ", np.min(geo_dist_svm_m))

Average SVM error [m] = 277.46583
Max SVM error [m] =  6403.985884763381
Min SVM error [m] =  3.552153589000864


# Cross check

In [26]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/LTEM_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAXMM1,BMAXMM2,BMAXMM3,BMBACM1,BMBACM2,BMBAKN1,BMBAKN2,...,BMZVON2,BMZVON3,TBTS1M1,TBTS3M1,TBTS3M2,TBTS5N1,TBTS5N2,TBTS5N3,TBTS6M1,TBTS6M2
0,1701439521,49.227657,16.575590,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
1,1701439522,49.227656,16.575592,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
2,1701439523,49.227656,16.575593,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
3,1701439524,49.227654,16.575595,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
4,1701439525,49.227653,16.575596,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6518,1701447325,49.197592,16.608559,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6519,1701447326,49.197588,16.608551,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6520,1701447327,49.197582,16.608546,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6521,1701447328,49.197578,16.608546,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0


In [27]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]
y_check_ori = cross_check_df.iloc[:, 1:3]

In [28]:
# Scaling cross check data

sc_X_cross = StandardScaler()
sc_y_cross = StandardScaler()
X_check = sc_X_cross.fit_transform(X_check)
y_check = sc_y_cross.fit_transform(y_check)

In [29]:
y_cross_lat = regressor_lat.predict(X_check)
y_cross_lon = regressor_lon.predict(X_check)

In [30]:
# R^2 score
score_lat_cross = r2_score(y_check[:,0], y_cross_lat)
print("R^2 lat cross check - {}%".format(round(score_lat_cross, 5) *100))

score_lon_cross = r2_score(y_check[:,1], y_cross_lon)
print("R^2 lon cross check - {}%".format(round(score_lon_cross, 5) *100))

score_combine_cross = (score_lat_cross + score_lon_cross)/2
print("R^2 combine cross check - {}%".format(round(score_combine_cross, 5) *100))

R^2 lat cross check - 78.179%
R^2 lon cross check - 54.712%
R^2 combine cross check - 66.445%


## MSE and RMSE

In [31]:
y_cross_join = np.stack((y_cross_lat, y_cross_lon), axis=1)
y_cross_join

array([[ 1.13696754, -0.64090325],
       [ 1.11682065, -0.63124807],
       [ 1.11682065, -0.63124807],
       ...,
       [-0.07035019, -0.09295444],
       [-0.07009796, -0.09224725],
       [-0.07020462, -0.09194935]])

In [32]:
y_cross_fin = sc_y_cross.inverse_transform(y_cross_join)
y_cross_fin

array([[49.21867063, 16.60152952],
       [49.21827575, 16.60172528],
       [49.21827575, 16.60172528],
       ...,
       [49.19500701, 16.61263925],
       [49.19501196, 16.61265359],
       [49.19500987, 16.61265963]])

In [33]:
y_cross_fin.shape

(6523, 2)

In [34]:
mse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin)
rmse_svm_cross = mean_squared_error(y_check_ori, y_cross_fin, squared = False)

print(mse_svm_cross)
print(rmse_svm_cross)

0.0001350000030368686
0.011400123830034248


## Distance cross check

In [35]:
y_check_arr = y_check_ori.to_numpy()

In [36]:
geo_dist_svm_cross = geo_distance(y_check_arr, y_cross_fin)
geo_dist_svm_cross

array([2.13241723, 2.16561603, 2.16555238, ..., 0.41284912, 0.4129125 ,
       0.41328935])

In [37]:
geo_dist_svm_cross_m = [i * 1000 for i in geo_dist_svm_cross]
geo_dist_svm_cross_m

[2132.417228834966,
 2165.616030917534,
 2165.552384632886,
 2131.940189082908,
 2131.8239474274856,
 2131.579390579731,
 2164.7755394540022,
 2164.6482455511723,
 2164.510848917525,
 2164.3835623500036,
 2164.3098315282646,
 2164.1825539603274,
 2164.1189154927424,
 2164.055277236104,
 2163.9916412728585,
 2163.8744432500357,
 2163.6936066467774,
 2130.1279526246185,
 2129.947549275123,
 2163.2147501502686,
 2163.1511082093784,
 2129.7029927852864,
 2162.970275716937,
 2129.53464075934,
 2129.4183905305485,
 2129.3142182977685,
 2129.3262983392756,
 2129.2742211891027,
 2129.170086348838,
 2129.0659675811094,
 2128.9618670049667,
 2128.921962762871,
 2128.8699265244923,
 2128.8820810572797,
 2128.8300586618066,
 2162.1980466820164,
 2162.2082506799507,
 2128.9185802237685,
 2128.9827649975605,
 2128.9307619961733,
 2129.0469499529695,
 2162.4629452905288,
 2129.227325902718,
 2162.7608828028615,
 2162.824557061795,
 2197.14226559373,
 2163.1224968491633,
 2129.7806321032563,
 2129.844

In [38]:
average_error_svm_cross = Average(geo_dist_svm_cross_m)

# Printing average of the list
print("Average SVM error cross check [m] =", round(average_error_svm_cross, 5))
print("Max SVM error cross check [m] = ", np.max(geo_dist_svm_cross_m))
print("Min SVM error cross check [m] = ", np.min(geo_dist_svm_cross_m))

Average SVM error cross check [m] = 1221.72296
Max SVM error cross check [m] =  4806.752373694748
Min SVM error cross check [m] =  18.661246812508203
